<a href="https://colab.research.google.com/github/hersheyyyy/Assessment-/blob/main/Alexey.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import Layout, HBox, VBox

# ---- Constants ----
E = 169e9        # Young’s modulus (Pa)
rho = 2330       # Density (kg/m^3)
A_drive = 1e-6   # Drive amplitude (m)
V_bias = 1.0     # Bias voltage (V)
Q = 1e4          # Quality factor
w_beam = 6.5e-6  # Beam width (m)
gap = 10e-6      # Electrode gap (m)
n_beams = 8      # Number of beams
Ri = 2400e-6     # Inner radius fixed (m)

# ---- Heading ----
heading = widgets.HTML(
    value="<h2 style='background-color: yellow; color: black; padding: 10px; text-align:center;'>Harshitha's Ring MEMS Gyroscope</h2>"
)

# ---- Function to create black-text boxes ----
def create_black_box(value, width='80px'):
    box = widgets.Text(
        value=value,
        disabled=True,
        layout=Layout(width=width, height='30px'),
        style={'description_width':'0px'}
    )
    box.add_class('black-text-box')
    return box

# ---- Input and output boxes ----
radius_number_box = create_black_box('3000', '80px')
radius_unit_box = create_black_box('µm', '50px')
thickness_number_box = create_black_box('100', '80px')
thickness_unit_box = create_black_box('µm', '50px')
sensitivity_box = create_black_box('', '180px')

# ---- Sliders ----
ring_radius_slider = widgets.IntSlider(
    value=3000, min=2500, max=3500, step=10,
    description='', continuous_update=True, readout=False,
    style={'bar_color':'blue'}
)
thickness_slider = widgets.IntSlider(
    value=100, min=50, max=150, step=5,
    description='', continuous_update=True, readout=False,
    style={'bar_color':'blue'}
)

# ---- Update function ----
def update_values(change):
    # Update number boxes
    radius_number_box.value = str(ring_radius_slider.value)
    thickness_number_box.value = str(thickness_slider.value)

    # Derived geometry
    Ro = ring_radius_slider.value * 1e-6
    h = thickness_slider.value * 1e-6
    L_beam = Ri
    area_ring = np.pi * (Ro**2 - Ri**2)
    m_ring = rho * area_ring * h

    # Stiffness & sensitivity
    k_beam = E * w_beam * (h**3) / (L_beam**3)
    k_total = n_beams * k_beam
    omega0 = np.sqrt(k_total / (0.5*m_ring))
    Gain = Q * (2 * 0.5*m_ring * A_drive * omega0) / k_total
    Sensitivity = V_bias * Gain / gap
    sensitivity_box.value = f"{Sensitivity:.6e} V/(rad/s)"

# Observe sliders
ring_radius_slider.observe(update_values, names='value')
thickness_slider.observe(update_values, names='value')

# ---- Layout ----
radius_box_row = HBox([radius_number_box, radius_unit_box])
thickness_box_row = HBox([thickness_number_box, thickness_unit_box])

input_box = VBox([
    widgets.HTML("<b>Input Data</b>"),
    widgets.Label("Ring Radius"),
    radius_box_row,
    ring_radius_slider,
    widgets.Label("Thickness"),
    thickness_box_row,
    thickness_slider
])

target_box = VBox([
    widgets.HTML("<b>Target Specs</b>"),
    widgets.Label("Mechanical Sensitivity"),
    sensitivity_box
])

ui = HBox([input_box, widgets.Label("   "), target_box], layout=Layout(justify_content='center'))

# ---- Display ----
display(heading, ui)

# ---- CSS for black text inside boxes ----
display(widgets.HTML('''
<style>
.black-text-box input {
    color: black !important;
    font-weight: bold;
}
</style>
'''))

# Initial calculation
update_values(None)


HTML(value="<h2 style='background-color: yellow; color: black; padding: 10px; text-align:center;'>Harshitha's …

HTML(value='\n<style>\n.black-text-box input {\n    color: black !important;\n    font-weight: bold;\n}\n</sty…

In [ ]:
import numpy as np
import pandas as pd

# ---- Constants ----
E = 169e9        # Young's modulus of silicon (Pa)
rho = 2330       # Density of silicon (kg/m^3)
A_drive = 1e-6   # Drive amplitude (m)
V_bias = 1.0     # Bias voltage (V)
Q = 1e4          # Quality factor

# ---- Resonator design parameters ----
D_outer = 6080e-6   # Outer diameter (m)
d_inner = 2400e-6   # Inner diameter (m)
h_struct = 100e-6   # Structure height (m)
w_beam = 6.5e-6     # Beam width (m)
gap = 10e-6         # Electrode gap (m)
n_beams = 8         # Number of beams

# Fixed inner radius
Ri = d_inner / 2
Ro = D_outer / 2

# ---- Function to calculate gyro parameters ----
def gyro_params(Ro, h):
    """
    Calculate natural frequency, mechanical gain, and electrical sensitivity
    Ro: outer radius [m]
    h: structure height/thickness [m]
    Returns: f0 [Hz], Gain [m/(rad/s)], Sensitivity [V/(rad/s)]
    """
    # Ring area & effective mass
    area_ring = np.pi * (Ro**2 - Ri**2)
    m_ring = rho * area_ring * h
    m_eff = 0.5 * m_ring  # effective mass

    # Beam stiffness
    L_beam = Ri  # assume beam length ≈ inner radius
    k_beam = E * w_beam * (h**3) / (L_beam**3)
    k_total = n_beams * k_beam

    # Natural frequency
    f0 = (1/(2*np.pi)) * np.sqrt(k_total / m_eff)

    # Mechanical gain
    omega0 = 2 * np.pi * f0
    Gain = Q * (2 * m_eff * A_drive * omega0) / k_total

    # Electrical sensitivity
    Sensitivity = V_bias * Gain / gap

    return f0, Gain, Sensitivity

# ---- 1. Single parameter set ----
f0_single, Gain_single, Sens_single = gyro_params(Ro, h_struct)
print("Single Parameter Set:")
print(f"Natural frequency f0 = {f0_single:.2f} Hz")
print(f"Mechanical gain = {Gain_single:.3e} m/(rad/s)")
print(f"Electrical sensitivity = {Sens_single:.3e} V/(rad/s)")

# ---- 2. Parameter sweep ----
radius_range = np.linspace(2500e-6, 3500e-6, 11)  # 2500–3500 µm
thickness_range = np.linspace(50e-6, 150e-6, 5)   # 50–150 µm

sweep_results = []

for h in thickness_range:
    for R in radius_range:
        f0, Gain, Sensitivity = gyro_params(R, h)
        sweep_results.append({
            'Radius_um': R*1e6,
            'Thickness_um': h*1e6,
            'f0_Hz': f0,
            'Gain_m_per_rad_s': Gain,
            'Sensitivity_V_per_rad_s': Sensitivity
        })

df_sweep = pd.DataFrame(sweep_results)

# Print first 10 rows
print("\nParameter Sweep (first 10 rows):")
print(df_sweep.head(10).to_string(index=False))



Single Parameter Set:
Natural frequency f0 = 6716.82 Hz
Mechanical gain = 4.739e-07 m/(rad/s)
Electrical sensitivity = 4.739e-02 V/(rad/s)

Parameter Sweep (first 10 rows):
 Radius_um  Thickness_um       f0_Hz  Gain_m_per_rad_s  Sensitivity_V_per_rad_s
    2500.0          50.0 4277.133972      7.442130e-07                 0.074421
    2600.0          50.0 4066.956955      7.826734e-07                 0.078267
    2700.0          50.0 3878.354106      8.207345e-07                 0.082073
    2800.0          50.0 3707.963112      8.584494e-07                 0.085845
    2900.0          50.0 3553.112864      8.958620e-07                 0.089586
    3000.0          50.0 3411.650185      9.330086e-07                 0.093301
    3100.0          50.0 3281.817026      9.699197e-07                 0.096992
    3200.0          50.0 3162.161567      1.006621e-06                 0.100662
    3300.0          50.0 3051.472692      1.043135e-06                 0.104314
    3400.0          50.0 29

In [ ]:
import numpy as np

# ---- Given design parameters (table) ----
D_outer = 6080e-6    # outer diameter (m)
d_inner = 2400e-6    # inner diameter (m)
h = 100e-6           # structure height / thickness (m)
w_beam = 6.5e-6      # beam width (m)
gap = 10e-6          # electrode gap (m)
n_beams = 8          # number of beams

# ---- Material & operational constants ----
E = 169e9            # Young's modulus of silicon (Pa)
rho = 2328.3           # density of silicon (kg/m^3)
A_drive = 1e-6       # drive amplitude (m)
V_bias = 1.0         # bias voltage (V)
Q = 1e4              # quality factor (dimensionless)

# ---- Derived geometry ----
Ri = d_inner / 2.0
Ro = D_outer / 2.0

# ---- Compute ring mass and effective mass ----
area_ring = np.pi * (Ro**2 - Ri**2)       # cross-sectional area of ring (m^2)
m_ring = rho * area_ring * h              # total mass of ring (kg)
m_eff = 0.5 * m_ring                      # effective mass (kg) — assumption: mode factor 0.5

# ---- Beam stiffness (Euler-Bernoulli cantilever-like approx) ----
L_beam = Ri                               # assume beam length ≈ inner radius (m)
k_beam = E * w_beam * (h**3) / (L_beam**3)  # stiffness of one beam (N/m)
k_total = n_beams * k_beam                # total stiffness (N/m)

# ---- Natural frequency ----
omega0 = np.sqrt(k_total / m_eff)         # angular freq (rad/s)
f0 = omega0 / (2 * np.pi)                # natural frequency (Hz)

# ---- Mechanical sensitivity (gain) ----
# Using the resonant amplification expression:
# Gain (m per (rad/s)) = Q * (2 * m_eff * A_drive * omega0) / k_total
Gain = Q * (2.0 * m_eff * A_drive * omega0) / k_total

# ---- Electrical sensitivity (V per (rad/s)) ----
Sensitivity = V_bias * Gain / gap

# ---- Print results with units ----
print("Calculated target gyro parameters (single parameter set):")
print("-------------------------------------------------------")
print(f"Outer radius Ro = {Ro*1e6:.1f} µm, Inner radius Ri = {Ri*1e6:.1f} µm, thickness h = {h*1e6:.1f} µm")
print()
print(f"Total ring area (m^2) = {area_ring:.6e}")
print(f"Total ring mass m_ring (kg) = {m_ring:.6e}")
print(f"Effective mass m_eff (kg) = {m_eff:.6e}")
print(f"Total stiffness k_total (N/m) = {k_total:.6e}")
print()
print(f"Natural frequency f0 = {f0:.2f} Hz")
print(f"Mechanical sensitivity (Gain) = {Gain:.6e} m/(rad/s)")
print(f"Electrical sensitivity = {Sensitivity:.6e} V/(rad/s)")


Calculated target gyro parameters (single parameter set):
-------------------------------------------------------
Outer radius Ro = 3040.0 µm, Inner radius Ri = 1200.0 µm, thickness h = 100.0 µm

Total ring area (m^2) = 2.450945e-05
Total ring mass m_ring (kg) = 5.706535e-06
Effective mass m_eff (kg) = 2.853268e-06
Total stiffness k_total (N/m) = 5.085648e+03

Natural frequency f0 = 6719.27 Hz
Mechanical sensitivity (Gain) = 4.737269e-07 m/(rad/s)
Electrical sensitivity = 4.737269e-02 V/(rad/s)
